In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from mlxtend.frequent_patterns import apriori, association_rules

**Association analysis shows which item is frequently purchased with other items.**

In [2]:
# !conda install --yes --prefix {sys.prefix} numpy

In [3]:
# conda install -c conda-forge mlxtend

In [4]:
data = pd.read_excel('SalesReport.xls')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [5]:
data.describe()

,Row ID,Postal Code,Sales,Quantity,Discount,Profit
count,9994.000000,9983.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,55245.233297,229.858001,3.789574,0.156203,28.656896
std,2885.163629,32038.715955,623.245101,2.225110,0.206452,234.260108
min,1.000000,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,57103.000000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,90008.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,99301.000000,22638.480000,14.000000,0.800000,8399.976000


In [6]:
data.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9983 non-null   float64       
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [8]:
data['State'].unique()

array(['Kentucky', 'California', 'Florida', 'North Carolina',
       'Washington', 'Texas', 'Wisconsin', 'Utah', 'Nebraska',
       'Pennsylvania', 'Illinois', 'Minnesota', 'Michigan', 'Delaware',
       'Indiana', 'New York', 'Arizona', 'Virginia', 'Tennessee',
       'Alabama', 'South Carolina', 'Oregon', 'Colorado', 'Iowa', 'Ohio',
       'Missouri', 'Oklahoma', 'New Mexico', 'Louisiana', 'Connecticut',
       'New Jersey', 'Massachusetts', 'Georgia', 'Nevada', 'Rhode Island',
       'Mississippi', 'Arkansas', 'Montana', 'New Hampshire', 'Maryland',
       'District of Columbia', 'Kansas', 'Vermont', 'Maine',
       'South Dakota', 'Idaho', 'North Dakota', 'Wyoming',
       'West Virginia'], dtype=object)

In [9]:
basket_california = (data[data['State'] == "California"]
                .groupby(['Order ID', 'Product Name'])["Quantity"]
                .sum().unstack().reset_index().fillna(0)
                .set_index('Order ID'))

In [10]:
basket_new_york = (data[data['State'] == "New York"]
                .groupby(['Order ID', 'Product Name'])["Quantity"]
                .sum().unstack().reset_index().fillna(0)
                .set_index('Order ID'))

In [11]:
basket_texas = (data[data['State'] == "Texas"]
                .groupby(['Order ID', 'Product Name'])["Quantity"]
                .sum().unstack().reset_index().fillna(0)
                .set_index('Order ID'))

In [12]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [19]:
basket_sets = basket_california.applymap(encode_units)

frequent_itemsets = apriori(basket_sets, min_support=0.001, use_colnames=True)
rules_california = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [20]:
rules_california

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Microsoft Natural Ergonomic Keyboard 4000),(Boston 16765 Mini Stand Up Battery Pencil Sha...,0.002938,0.001959,0.001959,0.666667,340.333333,0.001953,2.994123
1,(Boston 16765 Mini Stand Up Battery Pencil Sha...,(Microsoft Natural Ergonomic Keyboard 4000),0.001959,0.002938,0.001959,1.000000,340.333333,0.001953,inf


In [32]:
rules_california[rules_california['antecedents'] == {'Microsoft Natural Ergonomic Keyboard 4000'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Microsoft Natural Ergonomic Keyboard 4000),(Boston 16765 Mini Stand Up Battery Pencil Sha...,0.002938,0.001959,0.001959,0.666667,340.333333,0.001953,2.994123


**For example, Microsoft Natural Ergonomic Keyboard 4000 is frequently purchased with Boston 16765 Mini Stand Up Battery Pencil Sharpener in California.**

In [23]:
basket_sets = basket_new_york.applymap(encode_units)

frequent_itemsets = apriori(basket_sets, min_support=0.001, use_colnames=True)
rules_new_york = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [24]:
rules_new_york

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Adams Phone Message Book, 200 Message Capacit...","(#10 Gummed Flap White Envelopes, 100/Box)",0.005338,0.003559,0.001779,0.333333,93.666667,0.001760,1.494662
1,"(#10 Gummed Flap White Envelopes, 100/Box)","(Adams Phone Message Book, 200 Message Capacit...",0.003559,0.005338,0.001779,0.500000,93.666667,0.001760,1.989324
2,"(Dixon Prang Watercolor Pencils, 10-Color Set ...","(#10 Gummed Flap White Envelopes, 100/Box)",0.005338,0.003559,0.001779,0.333333,93.666667,0.001760,1.494662
3,"(#10 Gummed Flap White Envelopes, 100/Box)","(Dixon Prang Watercolor Pencils, 10-Color Set ...",0.003559,0.005338,0.001779,0.500000,93.666667,0.001760,1.989324
4,(Sauder Barrister Bookcases),"(#10 Gummed Flap White Envelopes, 100/Box)",0.001779,0.003559,0.001779,1.000000,281.000000,0.001773,inf
...,...,...,...,...,...,...,...,...,...
5429413,"(Pressboard Covers with Storage Hooks, 9 1/2"" ...",(Logitech LS21 Speaker System - PC Multimedia ...,0.003559,0.001779,0.001779,0.500000,281.000000,0.001773,1.996441
5429414,"(SimpliFile Personal File, Black Granite, 15w ...",(Logitech LS21 Speaker System - PC Multimedia ...,0.001779,0.001779,0.001779,1.000000,562.000000,0.001776,inf
5429415,(Hon 4070 Series Pagoda Armless Upholstered St...,(Logitech LS21 Speaker System - PC Multimedia ...,0.001779,0.001779,0.001779,1.000000,562.000000,0.001776,inf
5429416,(Apple iPhone 5),(Logitech LS21 Speaker System - PC Multimedia ...,0.003559,0.001779,0.001779,0.500000,281.000000,0.001773,1.996441


In [27]:
rules_new_york[rules_new_york['antecedents'] == {'Plastic Binding Combs'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
291,(Plastic Binding Combs),(Acme Preferred Stainless Steel Scissors),0.003559,0.003559,0.001779,0.5,140.5,0.001767,1.992883
493,(Plastic Binding Combs),"(Array Parchment Paper, Assorted Colors)",0.003559,0.003559,0.001779,0.5,140.5,0.001767,1.992883
1805,(Plastic Binding Combs),(Home/Office Personal File Carts),0.003559,0.001779,0.001779,0.5,281.0,0.001773,1.996441
1908,(Plastic Binding Combs),"(Howard Miller 16"" Diameter Gallery Wall Clock)",0.003559,0.001779,0.001779,0.5,281.0,0.001773,1.996441
1947,(Plastic Binding Combs),(Imation 8gb Micro Traveldrive Usb 2.0 Flash D...,0.003559,0.003559,0.001779,0.5,140.5,0.001767,1.992883
...,...,...,...,...,...,...,...,...,...
362839,(Plastic Binding Combs),"(Prang Dustless Chalk Sticks, Xerox 232, Imati...",0.003559,0.001779,0.001779,0.5,281.0,0.001773,1.996441
362900,(Plastic Binding Combs),"(Prang Dustless Chalk Sticks, Xerox 232, Home/...",0.003559,0.001779,0.001779,0.5,281.0,0.001773,1.996441
362962,(Plastic Binding Combs),"(Prang Dustless Chalk Sticks, Xerox 232, Imati...",0.003559,0.001779,0.001779,0.5,281.0,0.001773,1.996441
428372,(Plastic Binding Combs),"(Prang Dustless Chalk Sticks, Xerox 232, Imati...",0.003559,0.001779,0.001779,0.5,281.0,0.001773,1.996441


**Plastic Binding Combs is frequently purchased with Acme Preferred Stainless Steel Scissors and others more in New York.**

In [21]:
basket_sets = basket_texas.applymap(encode_units)

frequent_itemsets = apriori(basket_sets, min_support=0.001, use_colnames=True)
rules_texas = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [22]:
rules_texas

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Avery 499),"(#10- 4 1/8"" x 9 1/2"" Recycled Envelopes)",0.004107,0.002053,0.002053,0.500000,243.500000,0.002045,1.995893
1,"(#10- 4 1/8"" x 9 1/2"" Recycled Envelopes)",(Avery 499),0.002053,0.004107,0.002053,1.000000,243.500000,0.002045,inf
2,(Wasp CCD Handheld Bar Code Reader),"(#10- 4 1/8"" x 9 1/2"" Recycled Envelopes)",0.002053,0.002053,0.002053,1.000000,487.000000,0.002049,inf
3,"(#10- 4 1/8"" x 9 1/2"" Recycled Envelopes)",(Wasp CCD Handheld Bar Code Reader),0.002053,0.002053,0.002053,1.000000,487.000000,0.002049,inf
4,"(Eldon Expressions Desk Accessory, Wood Photo ...","(#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes)",0.004107,0.002053,0.002053,0.500000,243.500000,0.002045,1.995893
...,...,...,...,...,...,...,...,...,...
43589,"(Boston School Pro Electric Pencil Sharpener, ...","(Global Geo Office Task Chair, Gray, Xerox 199...",0.004107,0.002053,0.002053,0.500000,243.500000,0.002045,1.995893
43590,(Newell 348),"(Global Geo Office Task Chair, Gray, Xerox 199...",0.006160,0.002053,0.002053,0.333333,162.333333,0.002041,1.496920
43591,(Lunatik TT5L-002 Taktik Strike Impact Protect...,"(Global Geo Office Task Chair, Gray, Xerox 199...",0.002053,0.002053,0.002053,1.000000,487.000000,0.002049,inf
43592,(GBC Plastic Binding Combs),"(Global Geo Office Task Chair, Gray, Xerox 199...",0.004107,0.002053,0.002053,0.500000,243.500000,0.002045,1.995893


In [28]:
rules_texas[rules_texas['antecedents'] == {'Cisco SPA112 2 Port Phone Adapter'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
726,(Cisco SPA112 2 Port Phone Adapter),(Cisco SPA 502G IP Phone),0.004107,0.004107,0.002053,0.5,121.75,0.002037,1.991786
736,(Cisco SPA112 2 Port Phone Adapter),(DMI Arturo Collection Mission-style Design Wo...,0.004107,0.004107,0.002053,0.5,121.75,0.002037,1.991786
738,(Cisco SPA112 2 Port Phone Adapter),"(GE General Purpose, Extra Long Life, Showcase...",0.004107,0.004107,0.002053,0.5,121.75,0.002037,1.991786
740,(Cisco SPA112 2 Port Phone Adapter),(Permanent Self-Adhesive File Folder Labels fo...,0.004107,0.002053,0.002053,0.5,243.50,0.002045,1.995893
742,(Cisco SPA112 2 Port Phone Adapter),(Wireless Extenders zBoost YX545 SOHO Signal B...,0.004107,0.004107,0.002053,0.5,121.75,0.002037,1.991786
4195,(Cisco SPA112 2 Port Phone Adapter),"(Cisco SPA 502G IP Phone, GE General Purpose, ...",0.004107,0.002053,0.002053,0.5,243.50,0.002045,1.995893
4201,(Cisco SPA112 2 Port Phone Adapter),"(Cisco SPA 502G IP Phone, Permanent Self-Adhes...",0.004107,0.002053,0.002053,0.5,243.50,0.002045,1.995893
4207,(Cisco SPA112 2 Port Phone Adapter),"(Cisco SPA 502G IP Phone, Wireless Extenders z...",0.004107,0.002053,0.002053,0.5,243.50,0.002045,1.995893
4231,(Cisco SPA112 2 Port Phone Adapter),"(GE General Purpose, Extra Long Life, Showcase...",0.004107,0.002053,0.002053,0.5,243.50,0.002045,1.995893
4237,(Cisco SPA112 2 Port Phone Adapter),(Wireless Extenders zBoost YX545 SOHO Signal B...,0.004107,0.002053,0.002053,0.5,243.50,0.002045,1.995893


**Cisco SPA112 2 Port Phone Adapter is frequently purchased with Cisco SPA 502G IP Phone and others more in Texas.**